In [4]:
import sys
sys.path.append("../../")
sys.path.append("../../features")

from preprocessing import PreProcessing
from ExtractFeatures import extract_features
from features.GaborFilter.GaborFilter import generate_gabor_kernel_cv 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import skimage as ski
from sklearn.model_selection import train_test_split
import tqdm
import pandas as pd
import os
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score



Model Training Pipeline

In [3]:
# 1. Preprocess the images
# 2. Extract features
# 3. Train the model
# 4. Test the model

def main():

    folder_name = ["IBM Plex Sans Arabic" ,"Lemonada" , "Marhey" , "Scheherazade New" ]

    # for j in range(0 , 4 ) : 
    #     for i in tqdm(range(0, 1000)): 
    #         img = ski.io.imread("../../data/raw/fonts-dataset/"+folder_name[j]+"/"+str(i)+".jpeg")
            
    #         if len(np.unique(img)) == 1 : 
    #             continue # all image contain the same color 

    #         preprocessed_image = PreProcessing(img)
    #         ski.io.imsave("../../data/processed/fonts-dataset/"+folder_name[j]+"/"+str(i)+".jpeg", preprocessed_image)

    # 2. Extract features
    # Extracting features
    columns = ["area_ratio"]
    for j in range(1 , 17):
        columns.append("gabor_mean_" + str(j))
        columns.append("gabor_std_" + str(j))
        columns.append("gabor_energy_" + str(j))
    columns.append("font_type")
    features = pd.DataFrame(columns=columns)
    gabor_kernels = generate_gabor_kernel_cv()
    for j in range(0 , 4):
        for i in tqdm.tqdm(range(0, 1000)):
            pathname = "../../data/processed/fonts-dataset/"+folder_name[j]+"/"+str(i)+".jpeg"
            if os.path.exists(pathname):
                img = ski.io.imread(pathname)
                extractFeatures = extract_features(img,gabor_kernels)
                features = pd.concat([features , pd.DataFrame([np.concatenate([extractFeatures , [j]])] , columns = columns)])

    features.to_csv('./features_1000.csv')


  0%|          | 0/1000 [00:00<?, ?it/s]/mnt/6C7066F17066C186/spring24-4.2/pattern/project/Arabic-Font-Recognition/src/models/SVM/../../features/EnclosingArea/EnclosingArea.py:14: UserWarning: Any labeled images will be returned as a boolean array. Did you mean to use a boolean array?
  mask = morphology.remove_small_holes(image , area_threshold=10)
/tmp/ipykernel_104547/2649820595.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  features = pd.concat([features , pd.DataFrame([np.concatenate([extractFeatures , [j]])] , columns = columns)])
100%|██████████| 1000/1000 [04:36<00:00,  3.61it/s]


svm Accuracy: 0.2225


In [10]:


# 3. Train the model
dataset = pd.read_csv('./features_1000.csv')    
# column split
x_data = dataset.drop(columns=['Unnamed: 0'])     
x_data = x_data.iloc[:, :-1]                                   
y_data = dataset['font_type']
x_data, y_data = x_data.to_numpy(), y_data.to_numpy()

scaler = StandardScaler()
# Fit on training set only.
scaler.fit(x_data)
# Apply transform to both the training set and the test set.
x_data = scaler.transform(x_data)
#train-test split

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=75, stratify=y_data)
# Normalize the data

from sklearn.model_selection import GridSearchCV
from sklearn import svm

# Define the parameter grid
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4],
    'coef0': [0, 1, 2]
}

# Create the SVM model
svm_model = svm.SVC()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5)

# Fit the data to perform the hyperparameter search
grid_search.fit(x_train, y_train)

# Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

# Use the best model for prediction
y_pred = best_model.predict(x_val)
accuracy = accuracy_score(y_val, y_pred)
print("Best Model Accuracy:", accuracy)





In [8]:
# from sklearn.model_selection import cross_val_score

# dataset = pd.read_csv('./features_1000.csv')
# # column split
# x_data = dataset.drop(columns=['Unnamed: 0'])     
# x_data = x_data.iloc[:, :-1]                                   
# y_data = dataset['font_type']  

# scaler = StandardScaler()
# svm_model = svm.SVC(kernel='poly', C=10, gamma='scale', degree=3, coef0=1)
# # Fit on training set only.
# scaler.fit(x_data)

# #
# # Apply transform to both the training set and the test set.
# x_data = scaler.transform(x_data)
# scores = cross_val_score(svm_model, x_data, y_data, cv=5)

# # Print the accuracy for each fold
# print("Scores:", scores)

# # And the mean accuracy of all 5 folds
# print("Mean:", scores.mean())

# #train-test split
# # x_data, y_data = x_data.to_numpy(), y_data.to_numpy()
# x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=0, stratify=y_data)


# svm_model = svm.SVC(kernel='poly', C=10, gamma='scale', degree=3, coef0=1)
# svm_model.fit(x_train, y_train)
# # 4. Test the model
# y_pred = svm_model.predict(x_val)
# accuracy = accuracy_score(y_val, y_pred)
# print("svm Accuracy:", accuracy)